In [50]:
from collections import namedtuple

# Computational Model: DFA
DFA = namedtuple('DFA', ['T', 'S'])
BLANK, ZERO, ONE = ['.', '0', '1']

def encode(M):
    '''converts DFA model to binary representation'''
    code = ''

    for (current, symbol), successor in M.T.items():
        code += ''.join(list(map(lambda offset: ONE * (offset + 1) + ZERO,
                                [current, symbol, successor])))
    
    for final in M.S:
        code += ZERO
        code += ''.join(list(map(lambda offset: ONE * (offset + 1), [final])))

    return code

# DFA Representation
DFA.__repr__ = encode

In [51]:
'''Figure 6.3'''

# States
q0, q1, q2, q3 = 0, 1, 2, 3

# Transition Function
δ = dict([ 
    ((q0, 0), q0), ((q0, 1), q1),
    ((q1, 0), q1), ((q1, 1), q0),
])

XOR = DFA(δ, {q1})
print('XOR-DFA:', XOR)

XOR-DFA: 101010101101101101011011011010011


In [52]:
'''Figure 6.4'''

# Transition Function
δ = dict([
    ((q0, 0), q2), ((q0, 1), q3),
    ((q1, 0), q0), ((q1, 1), q3),
    ((q2, 0), q3), ((q2, 1), q1),
    ((q3, 0), q3), ((q3, 1), q3),
])

F = DFA(δ, {q0})
print('F-DFA:', F)

F-DFA: 101011101011011110110101011011011110111010111101110110110111101011110111101101111001


In [53]:
M = XOR

alphabet = {0, 1}
state = sorted(list(set(([x[0] for x in M.T.keys()]))))
current = state[0]
final = M.S

print(state, alphabet, current, final, sep=', ')

[0, 1], {0, 1}, 0, {1}


In [54]:
_input = r'101101'

def compute(code):
    global current
    current = state[0]

    X = [int(c) for c in code]

    for symbol in X:
        current = M.T[(current, symbol)]
        print('q' + str(current), '-> ', end='')

    return 'accept' if current in final else 'reject'


print(compute(_input))

q1 -> q1 -> q0 -> q1 -> q1 -> q0 -> reject
